<h1>Stacking Attempt</h1>

In [2]:
from sales_forecast import forecast_model
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook
sns.set_style("whitegrid")
sns.set_palette("colorblind")

from sales_forecast import forecast_model


In [3]:
model = forecast_model()

The following code is Martin's work, didn't touch it.

In [4]:
# This is a core part of the transition from a time-series prediction to a regression problem.
# The main idea is to featurize time in a way that it will keep the important information about the date.

def generate_features(df, sku_features, id_map):
        
    # Add metadata
    df = pd.merge(df, id_map, how='left', on='ts_id')
    df = pd.merge(df, sku_features, how='left', on='SKU')

    # Time features
    df['day_of_month'] = df['Date'].dt.day
    df['day_of_week'] = df['Date'].dt.dayofweek
    df['month'] = df['Date'].dt.month
    df['year'] = df['Date'].dt.year
    df['week'] = df['Date'].dt.week
    
    # Enlarge promo features
    # Since we know that promo is important
    
    #df['ts_promo'] = df['ts_id'].astype(str) + df['isPromo'].astype(str)
    #df['store_promo'] = df['Store'].astype(str) + df['isPromo'].astype(str)
    #df['segment_promo'] = df['Segment'].astype(str) + df['isPromo'].astype(str)
    #df['brand_promo'] = df['Brand'].astype(str) + df['isPromo'].astype(str)
    #df['sku_promo'] = df['SKU'].astype(str) + df['isPromo'].astype(str)
    
    #df['dom_promo'] = df['day_of_month'].astype(str) + df['isPromo'].astype(str)
    #df['dow_promo'] = df['day_of_week'].astype(str) + df['isPromo'].astype(str)
    
    return df

In [5]:
# Generate 2 dataframes
train_with_features = generate_features(model.train_data, model.features, model.stores)
test_with_features = generate_features(model.test_data, model.features, model.stores)

In [6]:
train_encoded = pd.get_dummies(train_with_features, columns = ['isPromo',
                                                               'Segment',
                                                               'Pack',
                                                               'Product'])


# Modify test dataset
test_encoded = pd.get_dummies(test_with_features, columns = ['isPromo',
                                                               'Segment',
                                                               'Pack',
                                                               'Product'])


In [7]:
#Delete columns from the encoded dataset: Some of these should go back in
del(train_encoded['SKU'], train_encoded['Store'], train_encoded['Brand'], train_encoded['Volume'],train_encoded['day_of_month'])
del(test_encoded['SKU'], test_encoded['Store'], test_encoded['Brand'], test_encoded['Volume'],test_encoded['day_of_month'])

In [8]:
train_y = train_encoded['Sales']
train_x = train_encoded
del(train_x['Sales'], train_x['Date'])
# I have to remove SKU column
#del(train_x['SKU'])

In [9]:
test = test_encoded
del(test['Date'], test['id']) # Should I delete id?? 

In [10]:
print(train_x.columns)
print(test.columns)

Index(['ts_id', 'day_of_week', 'month', 'year', 'week', 'isPromo_0.0',
       'isPromo_1.0', 'isPromo_2.0', 'Segment_High Margin',
       'Segment_MainStream', 'Segment_Premium', 'Pack_Bottle', 'Pack_Can',
       'Pack_PET', 'Product_Energy-drink', 'Product_Lager', 'Product_Non-alco',
       'Product_Other'],
      dtype='object')
Index(['ts_id', 'day_of_week', 'month', 'year', 'week', 'isPromo_0.0',
       'isPromo_1.0', 'isPromo_2.0', 'Segment_High Margin',
       'Segment_MainStream', 'Segment_Premium', 'Pack_Bottle', 'Pack_Can',
       'Pack_PET', 'Product_Energy-drink', 'Product_Lager', 'Product_Non-alco',
       'Product_Other'],
      dtype='object')


Let's import catboost.

The idea is to create new features : we train a model on 80% of the training data to predict the other 20%, and then add it as a column to make the LGBM regressor work on it.

In [11]:
def sliding_feature_stacking(data, sales, test, booster, num_samples=5):
    sales_predicted = []
    prop_train_test = 1 / num_samples
    
    prediction_for_test_set = [[] for _ in range(num_samples)]
    
    for i in range(num_samples):
        test_data = data[int(i * data.shape[0] * prop_train_test) : int((i + 1) * data.shape[0] * prop_train_test)]
        test_sales = sales[int(i * data.shape[0] * prop_train_test) : int((i + 1) * data.shape[0] * prop_train_test)]
        train_data = data.drop(list(range(int(i * data.shape[0] * prop_train_test), int((i + 1) * data.shape[0] * prop_train_test))))
        train_sales = sales.drop(list(range(int(i * data.shape[0] * prop_train_test), int((i + 1) * data.shape[0] * prop_train_test))))

        booster.fit(train_data, train_sales)
        prediction = booster.predict(test_data).flatten()
        sales_predicted += list(prediction)
        
        prediction_for_test_set[i] = booster.predict(test).flatten()

        print(f"{i + 1}/{num_samples}")
        
    prediction_for_test_set = np.array(prediction_for_test_set)
    return sales_predicted, np.average(prediction_for_test_set, axis=0)

In [12]:
from catboost import CatBoost

cat_booster = CatBoost(params={'iterations':100, 'learning_rate':0.5, 'grow_policy':'SymmetricTree'})

predicted_catboost_feature, predicted_catboost_for_test = sliding_feature_stacking(train_x, train_y, test, cat_booster)

0:	learn: 0.4326015	total: 255ms	remaining: 25.2s
1:	learn: 0.4177612	total: 409ms	remaining: 20s
2:	learn: 0.4122069	total: 603ms	remaining: 19.5s
3:	learn: 0.4095224	total: 757ms	remaining: 18.2s
4:	learn: 0.4075141	total: 924ms	remaining: 17.5s
5:	learn: 0.4065054	total: 1.08s	remaining: 16.9s
6:	learn: 0.4055604	total: 1.25s	remaining: 16.7s
7:	learn: 0.4047544	total: 1.4s	remaining: 16.1s
8:	learn: 0.4040557	total: 1.56s	remaining: 15.8s
9:	learn: 0.4032925	total: 1.71s	remaining: 15.4s
10:	learn: 0.4027212	total: 1.89s	remaining: 15.3s
11:	learn: 0.4022537	total: 2.02s	remaining: 14.8s
12:	learn: 0.4018271	total: 2.17s	remaining: 14.6s
13:	learn: 0.4014329	total: 2.32s	remaining: 14.3s
14:	learn: 0.4010415	total: 2.49s	remaining: 14.1s
15:	learn: 0.4005444	total: 2.63s	remaining: 13.8s
16:	learn: 0.4001828	total: 2.8s	remaining: 13.7s
17:	learn: 0.3994643	total: 2.96s	remaining: 13.5s
18:	learn: 0.3993095	total: 3.13s	remaining: 13.3s
19:	learn: 0.3985008	total: 3.3s	remaining: 1

63:	learn: 0.4022995	total: 11.8s	remaining: 6.63s
64:	learn: 0.4020518	total: 11.9s	remaining: 6.43s
65:	learn: 0.4009534	total: 12.1s	remaining: 6.25s
66:	learn: 0.4008366	total: 12.3s	remaining: 6.05s
67:	learn: 0.4004280	total: 12.4s	remaining: 5.85s
68:	learn: 0.4002177	total: 12.6s	remaining: 5.67s
69:	learn: 0.4000651	total: 12.8s	remaining: 5.48s
70:	learn: 0.3998189	total: 13s	remaining: 5.29s
71:	learn: 0.3997522	total: 13.1s	remaining: 5.11s
72:	learn: 0.3995845	total: 13.3s	remaining: 4.92s
73:	learn: 0.3993495	total: 13.5s	remaining: 4.74s
74:	learn: 0.3992175	total: 13.7s	remaining: 4.56s
75:	learn: 0.3991459	total: 13.9s	remaining: 4.37s
76:	learn: 0.3988972	total: 14s	remaining: 4.2s
77:	learn: 0.3988015	total: 14.2s	remaining: 4.01s
78:	learn: 0.3984193	total: 14.4s	remaining: 3.83s
79:	learn: 0.3980207	total: 14.6s	remaining: 3.65s
80:	learn: 0.3979263	total: 14.8s	remaining: 3.46s
81:	learn: 0.3977352	total: 14.9s	remaining: 3.28s
82:	learn: 0.3976220	total: 15.1s	re

27:	learn: 0.4184038	total: 5.08s	remaining: 13.1s
28:	learn: 0.4180860	total: 5.25s	remaining: 12.8s
29:	learn: 0.4176423	total: 5.44s	remaining: 12.7s
30:	learn: 0.4171580	total: 5.62s	remaining: 12.5s
31:	learn: 0.4165474	total: 5.8s	remaining: 12.3s
32:	learn: 0.4162669	total: 6s	remaining: 12.2s
33:	learn: 0.4155061	total: 6.21s	remaining: 12.1s
34:	learn: 0.4152480	total: 6.38s	remaining: 11.9s
35:	learn: 0.4148666	total: 6.58s	remaining: 11.7s
36:	learn: 0.4146930	total: 6.74s	remaining: 11.5s
37:	learn: 0.4143071	total: 6.95s	remaining: 11.3s
38:	learn: 0.4138289	total: 7.12s	remaining: 11.1s
39:	learn: 0.4133948	total: 7.33s	remaining: 11s
40:	learn: 0.4127856	total: 7.51s	remaining: 10.8s
41:	learn: 0.4124947	total: 7.7s	remaining: 10.6s
42:	learn: 0.4123757	total: 7.87s	remaining: 10.4s
43:	learn: 0.4121386	total: 8.06s	remaining: 10.3s
44:	learn: 0.4119967	total: 8.22s	remaining: 10s
45:	learn: 0.4118251	total: 8.39s	remaining: 9.85s
46:	learn: 0.4115345	total: 8.56s	remain

90:	learn: 0.3805956	total: 16.4s	remaining: 1.62s
91:	learn: 0.3804926	total: 16.6s	remaining: 1.44s
92:	learn: 0.3804481	total: 16.7s	remaining: 1.26s
93:	learn: 0.3803973	total: 16.9s	remaining: 1.08s
94:	learn: 0.3803059	total: 17.1s	remaining: 898ms
95:	learn: 0.3801985	total: 17.2s	remaining: 718ms
96:	learn: 0.3801015	total: 17.4s	remaining: 538ms
97:	learn: 0.3800456	total: 17.6s	remaining: 358ms
98:	learn: 0.3800179	total: 17.7s	remaining: 179ms
99:	learn: 0.3799550	total: 17.9s	remaining: 0us
5/5


In [13]:
assert len(predicted_catboost_feature) == train_x.shape[0]

In [14]:
from xgboost import XGBRegressor

xgbr_booster = XGBRegressor(max_depth=2,
                    subsample=0.7,        # How many samples does it uses
                    n_estimators=500,     # Amount of trees generated
                    colsample_bytree=0.9, # How many features does it use
                    learning_rate=0.2)    # Setup size shrinkage

predicted_xgbr_feature, predicted_xgbr_for_test = sliding_feature_stacking(train_x, train_y, test, xgbr_booster)

1/5
2/5
3/5
4/5
5/5


In [15]:
#We add the predictions as a feature
train_x.insert(len(train_x.columns), "catboost_feature", predicted_catboost_feature)
train_x.insert(len(train_x.columns), "xgbrboost_feature", predicted_xgbr_feature)


In [16]:
print(train_x.columns)

Index(['ts_id', 'day_of_week', 'month', 'year', 'week', 'isPromo_0.0',
       'isPromo_1.0', 'isPromo_2.0', 'Segment_High Margin',
       'Segment_MainStream', 'Segment_Premium', 'Pack_Bottle', 'Pack_Can',
       'Pack_PET', 'Product_Energy-drink', 'Product_Lager', 'Product_Non-alco',
       'Product_Other', 'catboost_feature', 'xgbrboost_feature'],
      dtype='object')


In [17]:
train_x.insert(len(train_x.columns), "Sales", train_y)
model.train_features = train_x
model.test_features = test
model.test_features.insert(len(model.test_features.columns), "catboost_feature", predicted_catboost_for_test)
model.test_features.insert(len(model.test_features.columns), "xgbrboost_feature", predicted_xgbr_for_test)


In [18]:
model.run()

start filter
filter done


array([0.10283877, 0.76755095, 0.58232556, ..., 0.21761431, 0.54997162,
       0.23377907])